# Import

In [1]:
import PySimpleGUI as sg
import requests as rq 
import matplotlib.pyplot as plt
# from bs4 import BeautifulSoup as bs 
import csv
import xlsxwriter
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# import nltk
# from nltk import word_tokenize
# from nltk.corpus import stopwords
import string
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

# Data Crawling

In [ ]:
n = 1
workbook = xlsxwriter.Workbook('AllBookPage.xlsx')
worksheet = workbook.add_worksheet()
row = 1
for x in range(1000):
# Webpage url 
  url = 'https://sinta.ristekbrin.go.id/books?page='+str(x+1)
  page = rq.get(url)
  soup = bs(page.content, 'html.parser')

  # f = open('tableBookpage'+str(x+1)+'.csv', 'w') 
  # w = csv.writer(f)
  # w.writerow(('No','Judul')) 
  
  for link in soup.findAll("dt"):

    judul = link.find("h4",{"class": "uk-text-primary"}).text

    print(judul)

    worksheet.write(row, 0, n)
    worksheet.write(row, 1, judul)
    row += 1
    n+=1
  # f.close() #tutup file csv atau simpan
workbook.close()

# Data Preparation 1

In [2]:
#Path jangan lupa diganti
df = pd.read_excel('AllBookPage.xlsx')
df.head()

,No,Judul
0,1,\t\r\nAplikasi Perkantoran Open Office Dan WPS...
1,2,\t\r\nICIIS 2019: Proceedings of the 2nd Inter...
2,3,\t\r\nKiat Sukses Meraih Hibah Penelitian Peng...
3,4,\t\r\nKOMPUTER TERAPAN JARINGAN SERTA PENGAPLI...
4,5,\t\r\nLoro Jonggrang Sci-fi


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9997 entries, 0 to 9996
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   No      9997 non-null   int64 
 1   Judul   9995 non-null   object
dtypes: int64(1), object(1)
memory usage: 156.3+ KB


In [4]:
df.isnull().sum()

No       0
Judul    2
dtype: int64

In [5]:
#clean data
df['Judul'] = df['Judul'].str.lower()
df['Judul'] = df['Judul'].str.replace('?','')
df['Judul'] = df['Judul'].str.replace('\t\r\n','')
df['Judul'] = df['Judul'].str.replace('\t','')
df = df.dropna(axis='rows')
df.isnull().sum()

No       0
Judul    0
dtype: int64

In [6]:
df.head()

,No,Judul
0,1,aplikasi perkantoran open office dan wps office
1,2,iciis 2019: proceedings of the 2nd internation...
2,3,kiat sukses meraih hibah penelitian pengembangan
3,4,komputer terapan jaringan serta pengaplikasiannya
4,5,loro jonggrang sci-fi


In [7]:
#most frequently called words
kata = df.Judul.str.split(expand=True).stack().value_counts()
kk_df = pd.DataFrame(kata)
kk_df.head()

,0
dan,2710
buku,2069
:,1449
ajar,1287
di,778


In [9]:
#split the words
mostfreq = df['make'] = df.Judul.str.split(' ').str[0]
mostfreq.head()
n = 100
mostfreq.value_counts()[:n].index.tolist()

['buku',
 'analisis',
 'dasar-dasar',
 'akuntansi',
 'aplikasi',
 'belajar',
 'dasar',
 'bahasa',
 'bunga',
 'asuhan',
 'bahan',
 'atlas',
 '',
 'cara',
 'dari',
 'biologi',
 'antologi',
 'aspek',
 'budaya',
 'a',
 'analisa',
 'basic',
 'budidaya',
 'administrasi',
 'bimbingan',
 'data',
 'algoritma',
 'arsitektur',
 'aksi',
 'bisnis',
 'catatan',
 'aku',
 'anatomi',
 'cerita',
 'aljabar',
 'bank',
 'asesmen',
 'anak',
 'agama',
 'alat',
 'biokimia',
 'covid-19',
 'book',
 'dampak',
 'business',
 'an',
 'bioteknologi',
 'dakwah',
 'corporate',
 'antropologi',
 'chapter',
 'audit',
 'cinta',
 '50',
 'biografi',
 'advanced',
 'bukan',
 'agribisnis',
 'aneka',
 'adaptasi',
 'batik',
 'ayo',
 'community',
 'cepat',
 'bermain',
 'bakteri',
 'artificial',
 'cakrawala',
 'ada',
 '100',
 'aktivitas',
 'apa',
 'best',
 '5',
 'berpikir',
 'advances',
 'basis',
 'air',
 'current',
 'asyiknya',
 'contemporary',
 'birokrasi',
 'character',
 '7',
 'anggaran',
 'cerdas',
 'collaborative',
 'bagaimana

In [10]:
#Path jangan lupa diganti
kata_df = pd.read_excel('kata_kunci.xlsx')
kata_dfArr = []
for index, row in kata_df.iterrows():
  kata_dfArr.append(row['kata_kunci'])
print(kata_dfArr)

['analisis', 'budaya', 'belajar', 'algoritma', 'asuhan', 'bahasa', 'atlas', 'anatomi', 'biologi', 'antologi', 'budidaya', 'bimbingan', 'arsitektur', 'aljabar', 'biokimia', 'bioteknologi', 'citra', 'assessment', 'antropologi', 'agribisnis', 'artificial', 'aplikasi', 'birokrasi', 'cerdas', 'academic', 'critical', 'autobiografi', 'biostatistik', 'antioksidan', 'aspek', 'pendidikan', 'desain', 'design', 'manajemen', 'e-commerce', 'hukum', 'entrepreneurship', 'design', 'komunikasi', 'buku', 'ilmu', 'implementasi', 'pembangunan', 'ekonomi', 'intelektual', 'konseling', 'praktik', 'sistem', 'observasi', 'filosofi', 'dasar-dasar', 'dasar', 'akuntansi', 'panduan', 'anak', 'kesehatan', 'pembelajaran', 'teori', 'bisnis', 'data', 'keperawatan', 'lingkungan', 'english', 'basic', 'practical', 'guide']


In [11]:
labelArr = []
for index, row in df.iterrows():
  array_kata = str(row['Judul']).split(' ')
  # print(array_kata)
  for i, kata in enumerate(array_kata):
    if kata in kata_dfArr:
      labelArr.append('Edukasi')
      # row['Label'] = ndf['Label'].str.replace('Null','Edukasi')
      print('Edukasi')
      break
    elif i == len(array_kata)-1 and kata not in kata_dfArr: 
      labelArr.append('Non Edukasi')
      # row['Label'] = ndf['Label'].str.replace('Null','Non Edukasi')
      print('Non Edukasi')
      break

Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Eduk

Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi


Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi


Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non 

Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi


Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi


Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Eduk

Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Non Edukasi
Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Edukasi
Edukasi
Edukasi
Non Edukasi
Non Edukasi
Non Eduk

In [12]:
df['Label'] = labelArr
df.head()

,No,Judul,make,Label
0,1,aplikasi perkantoran open office dan wps office,aplikasi,Edukasi
1,2,iciis 2019: proceedings of the 2nd internation...,iciis,Non Edukasi
2,3,kiat sukses meraih hibah penelitian pengembangan,kiat,Non Edukasi
3,4,komputer terapan jaringan serta pengaplikasiannya,komputer,Non Edukasi
4,5,loro jonggrang sci-fi,loro,Non Edukasi


In [ ]:
#save to csv
df.to_csv("data_labeling.csv") 

In [13]:
df['Label'].value_counts()

Edukasi        6459
Non Edukasi    3536
Name: Label, dtype: int64

# Data Preparation 2 + Klasifikasi


In [14]:
X = df['Judul']
y = df['Label']
print(X)

0         aplikasi perkantoran open office dan wps office
1       iciis 2019: proceedings of the 2nd internation...
2        kiat sukses meraih hibah penelitian pengembangan
3       komputer terapan jaringan serta pengaplikasiannya
4                                   loro jonggrang sci-fi
                              ...                        
9990             dataflow programming menggunakan labview
9991                                      datang ke dilam
9992                                datanglah kerajaan-mu
9993    dataran tinggi ijen potongan tanah surga untuk...
9994                                          datuk hitam
Name: Judul, Length: 9995, dtype: object


In [15]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.25,random_state=1)
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)

acc = 100*accuracy_score(ytest,ypred)

print('Accuracy %0.2f %%'%acc)
print("")
print(Xtest.shape)
print(Xtrain.shape)
print("")
print(ypred)

Accuracy 81.95 %

(2499,)
(7496,)

['Edukasi' 'Non Edukasi' 'Edukasi' ... 'Edukasi' 'Edukasi' 'Edukasi']


In [16]:
def prediksi_topik(kalimat, model):
    prediksi = model.predict([kalimat])
    return prediksi

In [17]:
contoh_judul_1 = "loro jonggrang sci-fi"
print(prediksi_topik(contoh_judul_1, model))

['Non Edukasi']


In [18]:
contoh_judul_2 = "App inventor: ciptakan sendiri aplikasi androidmu"
print(prediksi_topik(contoh_judul_2, model))

['Edukasi']


## GUI (use it properly)

In [19]:
sg.theme('Cyan')
   
layout = [[sg.Text('Tuliskan judul buku:'),
           sg.Text(size=(15,1), key='-OUTPUT-')],
          [sg.Input(key='-IN-')],
          [sg.Button('Tampilkan Hasil'), sg.Button('Exit')]]
  
window = sg.Window('Prediksi Topik Buku', layout)
  
while True:
    event, values = window.read()
    print(event, values)
    print(prediksi_topik(values['-IN-'], model))
    
    if event in  (None, 'Exit'):
        break
      
    if event == 'Tampilkan Hasil':
        # Update the "output" text element
        # to be the value of "input" element
        window['-OUTPUT-'].update(prediksi_topik(values['-IN-'], model))
  
window.close()

** Warning - Cyan Theme is not a valid theme. Change your theme call. **
valid values are ['Black', 'BlueMono', 'BluePurple', 'BrightColors', 'BrownBlue', 'Dark', 'Dark2', 'DarkAmber', 'DarkBlack', 'DarkBlack1', 'DarkBlue', 'DarkBlue1', 'DarkBlue10', 'DarkBlue11', 'DarkBlue12', 'DarkBlue13', 'DarkBlue14', 'DarkBlue15', 'DarkBlue16', 'DarkBlue17', 'DarkBlue2', 'DarkBlue3', 'DarkBlue4', 'DarkBlue5', 'DarkBlue6', 'DarkBlue7', 'DarkBlue8', 'DarkBlue9', 'DarkBrown', 'DarkBrown1', 'DarkBrown2', 'DarkBrown3', 'DarkBrown4', 'DarkBrown5', 'DarkBrown6', 'DarkBrown7', 'DarkGreen', 'DarkGreen1', 'DarkGreen2', 'DarkGreen3', 'DarkGreen4', 'DarkGreen5', 'DarkGreen6', 'DarkGreen7', 'DarkGrey', 'DarkGrey1', 'DarkGrey10', 'DarkGrey11', 'DarkGrey12', 'DarkGrey13', 'DarkGrey14', 'DarkGrey2', 'DarkGrey3', 'DarkGrey4', 'DarkGrey5', 'DarkGrey6', 'DarkGrey7', 'DarkGrey8', 'DarkGrey9', 'DarkPurple', 'DarkPurple1', 'DarkPurple2', 'DarkPurple3', 'DarkPurple4', 'DarkPurple5', 'DarkPurple6', 'DarkPurple7', 'DarkRe

TypeError: 'NoneType' object is not subscriptable